In [1]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import random

In [2]:
# VoTT のエクスポートファイルや、画像が格納されているディレクトリ
BASE_DIRECTORY = './vott-json-export/'
# VoTT のエクスポートファイル名
EXPORT_FILENAME = 'Machikado-export.json'
# 訓練データに使用する割合
TRAIN_RATIO = 0.8
# 乱数シード
RANDOM_STATE = 0

### DatasetCatalogを用意する

In [3]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from Machikado_vott import get_cat_names, get_machikado_dicts

# vott エクスポートファイルの読み込み
CAT_NAME2ID, CAT_ID2NAME = get_cat_names(os.path.join(BASE_DIRECTORY, EXPORT_FILENAME))
dataset_dicts = get_machikado_dicts(os.path.join(BASE_DIRECTORY, EXPORT_FILENAME), BASE_DIRECTORY, CAT_NAME2ID)

# 訓練用、テスト用に分ける
random.seed(RANDOM_STATE)
random.shuffle(dataset_dicts)

split_idx = int(len(dataset_dicts) * TRAIN_RATIO) + 1

# 登録
DatasetCatalog.clear()
DatasetCatalog.register('train', lambda : dataset_dicts[:split_idx])
DatasetCatalog.register('test', lambda : dataset_dicts[split_idx:])

MetadataCatalog.get('train').set(thing_classes=list(CAT_NAME2ID.keys()))
MetadataCatalog.get('test').set(thing_classes=list(CAT_NAME2ID.keys()))

警告: name: 59.jpg - 画像サイズの不整合 image_size:(268, 201), ./vott-json-export/Machikado-export.json: (600, 600)


Metadata(name='test', thing_classes=['Shamiko', 'Gosenzo', 'Lilith', 'Momo', 'Mikan', 'Mob'])

In [4]:
from detectron2.data import DatasetCatalog

# 訓練用、テスト用に分ける
random.seed(RANDOM_STATE)
random.shuffle(dataset_dicts)
split_idx = int(len(dataset_dicts) * TRAIN_RATIO) + 1

# 登録
DatasetCatalog.clear()
DatasetCatalog.register('train', lambda : dataset_dicts[:split_idx])
DatasetCatalog.register('test', lambda : dataset_dicts[split_idx:])

---
## 学習

In [5]:
from detectron2.config import get_cfg
from MachikadoDatasetMapper import MachikadoDatasetMapper, get_custom_cfg

cfg = get_cfg()
cfg.OUTPUT_DIR = './output'
cfg.CUDA = 'cuda:0'

# cfg.merge_from_file("../configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# cfg.MODEL.WEIGHTS = './coco_models/model_final_f10217.pkl'
# cfg.SOLVER.IMS_PER_BATCH = 2

# 重いけど、これ精度良いです。
cfg.merge_from_file('../configs/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml')
cfg.MODEL.WEIGHTS = './coco_models/model_final_2d9806.pkl'
cfg.SOLVER.IMS_PER_BATCH = 1 # GTX2070 ではこれが限界

cfg.DATASETS.TRAIN = ('train',)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1500    # 300 iterations seems good enough, but you can certainly train longer <- とあるが、まあデータセットによるよね・・・
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(CAT_ID2NAME) 

cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.CROP.SIZE = [0.8, 0.8]

custom_cfg = get_custom_cfg()

In [6]:
from MachikadoDatasetMapper import plot_dataset_mapper

# # random.seed(RANDOM_STATE)
# mapper = MachikadoDatasetMapper(cfg, custom_cfg=custom_cfg, is_train=True)
# plot_dataset_mapper(dataset_dicts, mapper)

In [7]:
from detectron2.engine import DefaultTrainer

from detectron2.data import (
    build_detection_test_loader,
    build_detection_train_loader,
)

class MachikadoTrainer(DefaultTrainer):
    def __init__(self, cfg):
        super().__init__(cfg)
        
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, MachikadoDatasetMapper(cfg, custom_cfg=custom_cfg))

In [ ]:
# 出力先のディレクトリを作る
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = MachikadoTrainer(cfg) 
trainer.resume_or_load(resume=False) # True で途中から学習できるらしい
trainer.train()

OK!!
[04/13 07:02:06 d2.data.detection_utils]: TransformGens used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[04/13 07:02:06 d2.data.build]: Removed 0 images with no usable annotations. 57 images left.
[04/13 07:02:06 d2.data.build]: Distribution of instances among all 6 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|  Shamiko   | 38           |  Gosenzo   | 20           |   Lilith   | 10           |
|    Momo    | 20           |   Mikan    | 8            |    Mob     | 8            |
|            |              |            |              |            |              |
|   total    | 104          |            |              |            |              |
[04/13 07:02:06 d2.data.common]: Serializing 57 elements to byte tensors and concatenating them all ...
[04

'roi_heads.box_predictor.cls_score.weight' has shape (81, 1024) in the checkpoint but (7, 1024) in the model! Skipped.
'roi_heads.box_predictor.cls_score.bias' has shape (81,) in the checkpoint but (7,) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.weight' has shape (320, 1024) in the checkpoint but (24, 1024) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.bias' has shape (320,) in the checkpoint but (24,) in the model! Skipped.
'roi_heads.mask_head.predictor.weight' has shape (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) in the model! Skipped.
'roi_heads.mask_head.predictor.bias' has shape (80,) in the checkpoint but (6,) in the model! Skipped.


[04/13 07:02:08 d2.engine.train_loop]: Starting training from iteration 0
[04/13 07:02:18 d2.utils.events]:  eta: 0:11:25  iter: 19  total_loss: 3.362  loss_cls: 1.877  loss_box_reg: 0.691  loss_mask: 0.693  loss_rpn_cls: 0.005  loss_rpn_loc: 0.022  time: 0.4677  data_time: 0.0234  lr: 0.000005  max_mem: 3671M
[04/13 07:02:27 d2.utils.events]:  eta: 0:11:11  iter: 39  total_loss: 3.132  loss_cls: 1.726  loss_box_reg: 0.627  loss_mask: 0.692  loss_rpn_cls: 0.010  loss_rpn_loc: 0.015  time: 0.4615  data_time: 0.0018  lr: 0.000010  max_mem: 3671M
[04/13 07:02:36 d2.utils.events]:  eta: 0:11:06  iter: 59  total_loss: 2.741  loss_cls: 1.390  loss_box_reg: 0.552  loss_mask: 0.688  loss_rpn_cls: 0.008  loss_rpn_loc: 0.023  time: 0.4619  data_time: 0.0018  lr: 0.000015  max_mem: 3765M
[04/13 07:02:46 d2.utils.events]:  eta: 0:10:56  iter: 79  total_loss: 2.277  loss_cls: 1.015  loss_box_reg: 0.491  loss_mask: 0.681  loss_rpn_cls: 0.005  loss_rpn_loc: 0.031  time: 0.4646  data_time: 0.0016  lr: